# Contrastive Loss (InfoNCE)

> Implements the InfoNCE loss function for CLIP training, handling distributed data parallel (DDP) correctly.

In [ ]:
#| default_exp loss

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

In [ ]:
#| export
try:
    import indic_clip.core
    print("Reloaded indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")
            # raise # Stop execution if core components missing

In [ ]:
#| hide
%cd /content/drive/MyDrive/Indic-Clip/

In [ ]:
#| hide
!pip install -qr requirements.txt

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import logging
from typing import Optional

from fastai.vision.all import *

try:
    from indic_clip.core import get_logger, setup_logging
except ModuleNotFoundError:
    # Fallback if core not found
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass

setup_logging()
logger = get_logger(__name__)

## AllGather Helper for DDP

In [ ]:
#| export
class AllGather(torch.autograd.Function):
    """Custom autograd function to gather tensors from all processes, supporting gradients."""

    @staticmethod
    def forward(ctx, tensor: torch.Tensor) -> torch.Tensor:
        """Performs the all_gather operation and prepares context for backward pass."""
        # Check if distributed environment is initialized
        if not dist.is_available() or not dist.is_initialized():
            # If not distributed, just return the input tensor
            return tensor

        # Ensure tensor is contiguous before gathering
        tensor = tensor.contiguous()
        world_size = dist.get_world_size()
        # Create a list to hold tensors from all ranks
        output = [torch.empty_like(tensor) for _ in range(world_size)]
        # Perform the all_gather operation
        dist.all_gather(output, tensor)
        # Concatenate the gathered tensors along the batch dimension (dim=0)
        gathered_tensor = torch.cat(output, dim=0)

        # Save world_size for backward pass (optional, could re-fetch)
        # ctx.world_size = world_size
        return gathered_tensor

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor) -> torch.Tensor:
        """Performs the reduce_scatter operation for the backward pass."""
        # Check if distributed environment is initialized
        if not dist.is_available() or not dist.is_initialized():
            # If not distributed, just return the gradient
            return grad_output

        logger.warning("!!! AllGather DEBUG: Returning SLICE of grad_output in backward pass !!!")
        # Ensure grad_output is contiguous
        grad_output = grad_output.contiguous()
        world_size = dist.get_world_size()

        # Check if the gradient tensor size is divisible by world_size
        if grad_output.shape[0] % world_size != 0:
            raise RuntimeError("Gradient output size must be divisible by world size for all_gather backward pass.")

        # Calculate the chunk size for each process
        chunk_size = grad_output.shape[0] // world_size

        # Prepare the input tensor for reduce_scatter (this will hold the gradient for the current rank)
        grad_input = torch.empty(chunk_size, *grad_output.shape[1:], dtype=grad_output.dtype, device=grad_output.device)

        # Perform reduce_scatter: sums gradients corresponding to each rank's input
        # The list comprehension splits the gathered gradient tensor back into chunks
        dist.reduce_scatter(grad_input, list(grad_output.chunk(world_size, dim=0)), op=dist.ReduceOp.SUM)

        # grad_input now contains the correct gradient sum for the input tensor on this rank
        return grad_input

## Contrastive Loss Implementation

In [ ]:
#| export
#| export
class ContrastiveLoss(Module): # Inherit from fastai's Module or nn.Module
    """Calculates the contrastive loss (InfoNCE) between image and text features.
    Handles distributed training by gathering features across GPUs before calculating loss.
    Designed to be used directly as Learner's loss_func.
    """
    def __init__(self):
        self.all_gather = AllGather.apply

    # *** IMPORTANT: Match fastai loss signature ***
    def forward(self, preds: tuple[torch.Tensor, torch.Tensor, torch.Tensor], *target) -> torch.Tensor:
        # preds is the output from the model: (image_features, text_features, logit_scale)
        # target is the dummy category batch, which we ignore here.
        # logger.critical("LOSS FORWARD CALLED") # Use critical for visibility

        image_features, text_features, logit_scale = preds
        device = image_features.device # Get the device from one of the inputs

        # --- Check features JUST BEFORE gathering/similarity calculation ---
        if torch.isnan(image_features).any():
            logger.error("!!! NaN DETECTED IN image_features JUST BEFORE ContrastiveLoss similarity calc !!!")
            return torch.tensor(torch.nan, device=device, requires_grad=True)
        if torch.isnan(text_features).any():
            logger.error("!!! NaN DETECTED IN text_features JUST BEFORE ContrastiveLoss similarity calc !!!")
            return torch.tensor(torch.nan, device=device, requires_grad=True)
        if torch.isnan(logit_scale):
            logger.error(f"!!! NaN DETECTED IN logit_scale JUST BEFORE ContrastiveLoss similarity calc: {logit_scale.item()} !!!")
            return torch.tensor(torch.nan, device=device, requires_grad=True)

        # --- Check norms AGAIN here (should be ~1.0) ---
        img_norms = image_features.norm(p=2, dim=-1)
        txt_norms = text_features.norm(p=2, dim=-1)
        if img_norms.min() < 0.9 or img_norms.max() > 1.1:
            logger.warning(f"Image feature norms deviate significantly from 1.0: min={img_norms.min().item()}, max={img_norms.max().item()}")
        if txt_norms.min() < 0.9 or txt_norms.max() > 1.1:
            logger.warning(f"Text feature norms deviate significantly from 1.0: min={txt_norms.min().item()}, max={txt_norms.max().item()}")

        # --- Gather Features Across GPUs (if applicable) ---
        if dist.is_available() and dist.is_initialized():
            logger.debug("Gathering features across GPUs.")
            gathered_image_features = self.all_gather(image_features)
            gathered_text_features = self.all_gather(text_features)
            # Logit scale is usually not gathered, assuming it's the same across devices
            # or handled differently (e.g., only rank 0 updates it).
        else:
            logger.debug("Not a distributed setup. Using local features.")
            gathered_image_features = image_features
            gathered_text_features = text_features
        # ---------------------------------------------------

        # --- Calculate Similarity ---
        # Note: logit_scale is used directly (already exponentiated by the model)
        logits_per_image = logit_scale * gathered_image_features @ gathered_text_features.t()
        # --- Check Logits Immediately ---
        if torch.isnan(logits_per_image).any() or torch.isinf(logits_per_image).any():
            logger.error("!!! NaN/Inf detected in logits_per_image AFTER similarity calculation !!!")
            logger.error(f"Logit Scale was: {logit_scale.item():.4f}")
            logger.error(f"Gathered Img Feat Stats: mean={gathered_image_features.mean().item()}, std={gathered_image_features.std().item()}, min={gathered_image_features.min().item()}, max={gathered_image_features.max().item()}")
            logger.error(f"Gathered Txt Feat Stats: mean={gathered_text_features.mean().item()}, std={gathered_text_features.std().item()}, min={gathered_text_features.min().item()}, max={gathered_text_features.max().item()}")
            return torch.tensor(torch.nan, device=device, requires_grad=True) # Return NaN

        logits_per_text = logits_per_image.t() # Efficient transpose

        # --- Create Labels ---
        # Ground truth is the identity matrix diagonal (image i matches text i)
        # The size should match the batch size *after* gathering
        gathered_batch_size = gathered_image_features.shape[0]
        labels = torch.arange(gathered_batch_size, device=device, dtype=torch.long)
        # ---------------------

        # --- Calculate Loss ---
        loss_img = F.cross_entropy(logits_per_image, labels)
        loss_txt = F.cross_entropy(logits_per_text, labels)

        # --- Check component losses ---
        if torch.isnan(loss_img): logger.error(f"!!! loss_img is NaN !!! Input logits max: {logits_per_image.max().item()}")
        if torch.isnan(loss_txt): logger.error(f"!!! loss_txt is NaN !!! Input logits max: {logits_per_text.max().item()}")

        total_loss = (loss_img + loss_txt) / 2

        # --- Final NaN check ---
        if torch.isnan(total_loss):
            logger.error(f"!!! total_loss is NaN !!! loss_img={loss_img.item()}, loss_txt={loss_txt.item()}")
            return torch.tensor(torch.nan, device=device, requires_grad=True) # Return NaN if components were NaN

        # logger.info(f"Returning total_loss: value={total_loss.item()}, shape={total_loss.shape}, dtype={total_loss.dtype}, device={total_loss.device}, requires_grad={total_loss.requires_grad}")
        return total_loss

## Example Usage and Testing

In [ ]:
#| eval: false
if __name__ == '__main__':
    print("--- Testing ContrastiveLoss (Non-Distributed) ---")
    # Setup dummy data
    batch_size = 4
    embed_dim = 512
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Simulate normalized features
    img_feat = F.normalize(torch.randn(batch_size, embed_dim, device=device), dim=-1)
    txt_feat = F.normalize(torch.randn(batch_size, embed_dim, device=device), dim=-1)
    # Simulate logit scale from model (already exponentiated)
    logit_scale = torch.exp(torch.ones([], device=device) * torch.log(torch.tensor(1/0.07)))

    print("Input Shapes:")
    print(f"  Image Features: {img_feat.shape}")
    print(f"  Text Features:  {txt_feat.shape}")
    print(f"  Logit Scale:    {logit_scale.shape}")

    # Instantiate loss
    loss_fn = ContrastiveLoss()

    # Calculate loss
    loss_val = loss_fn((img_feat, txt_feat, logit_scale))

    print(f"Output Loss: {loss_val.item():.4f} (Type: {type(loss_val)}, Device: {loss_val.device})")
    assert isinstance(loss_val, torch.Tensor) and loss_val.ndim == 0

    # --- Test Distributed Scenario (Simulated) ---
    print("\n--- Testing ContrastiveLoss (Simulated Distributed, World Size=2) ---")

    if dist.is_available() and not dist.is_initialized():
        # This part requires initializing a process group, usually done via torchrun/launch
        # We can't fully simulate it here without that setup.
        print("Distributed environment not available/initialized. Skipping DDP test.")
        print("Note: To run the distributed test, initialize a process group first.")
        print("Example (requires torchrun or similar):")
        print("  import torch.distributed as dist")
        print("  dist.init_process_group(backend='nccl') # Or 'gloo' for CPU")
        print("  # ... run the test code ...")

    elif dist.is_available() and dist.is_initialized():
        # This block would run if a process group *is* initialized
        world_size = dist.get_world_size()
        rank = dist.get_rank()
        print(f"Running DDP test on Rank {rank}/{world_size}")

        # Assume img_feat, txt_feat, logit_scale are the local tensors for this rank
        loss_val_ddp = loss_fn((img_feat, txt_feat, logit_scale))

        print(f"DDP Output Loss (Rank {rank}): {loss_val_ddp.item():.4f}")
        assert isinstance(loss_val_ddp, torch.Tensor) and loss_val_ddp.ndim == 0
        # Note: Loss value might differ across ranks if inputs are different,
        # but the calculation mechanism (gathering) is tested.
        # For identical inputs across ranks (less realistic), losses should match.

    else:
         print("Torch distributed is not available on this system.")

In [1]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export